In [26]:
import pandas as pd

In [32]:
class Datasets:
    
    def load_dataset(dataset_name):
        return pd.read_csv(f'../../datasets/anomaly_detection/{dataset_name}.csv')

class ATSFDataset:
    
    _data_df = None
    
    class Columns:
        TIMESTAMP = 'timestamp'
        VALUE_F = 'value_f'
        VALUE_C = 'value_c'
        
    
    def load(self):
        
        self._data_df = Datasets.load_dataset('ambient_temperature_system_failure')
        self._data_df[ATSFDataset.Columns.TIMESTAMP] = pd.to_datetime(self._data_df[ATSFDataset.Columns.TIMESTAMP])
        self._data_df.rename(columns={ 'value' : ATSFDataset.Columns.VALUE_F }, inplace=True, errors='raise')
        self._data_df[ATSFDataset.Columns.VALUE_C]= (self._data_df[ATSFDataset.Columns.VALUE_F]- 32) * 5 / 9
        
        return self
    
    def get_df(self):
        return self._data_df
    
#est = ATSFDataset().load()
#est.get_df().head()